In [36]:
import numpy as np
import torch

In [37]:
import pandas as pd

In [38]:
import torch
from torch import nn

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
import matplotlib as mpl

In [41]:
import matplotlib.pyplot as plt

In [42]:
CAD_path="/content/logs_CAD_2024_3_.xlsx"

In [43]:
data = pd.read_excel("/content/logs_CAD_2024_3_.xlsx")

In [44]:
display(pd.DataFrame(data))

,ID,Spec,MotherTongue,Gender,Age,Activity,DayPart,Lesson,Grade
0,29292,1,est,female,20.000,23,3,1,5
1,26864,2,rus,male,NaN,7,3,0,0
2,26861,2,rus,male,21.000,18,1,1,3
3,26760,3,est,female,NaN,8,1,1,5
4,26544,4,est,male,21.000,9,1,1,5
5,29598,11,est,male,25.000,7,1,1,0
6,29133,1,est,male,20.000,14,1,1,5
7,26604,2,rus,male,23.000,6,1,1,4
8,29346,1,est,male,20.000,4,1,1,5
9,26563,4,est,male,21.000,14,1,1,4


Settings

In [45]:
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [46]:
pd.options.display.float_format = '{:.3f}'.format

In [47]:
df = pd.read_excel("/content/logs_CAD_2024_3_.xlsx")
test_df = pd.read_excel("/content/logs_CAD_2024_3_.xlsx")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            17 non-null     int64  
 1   Spec          17 non-null     int64  
 2   MotherTongue  17 non-null     object 
 3   Gender        17 non-null     object 
 4   Age           13 non-null     float64
 5   Activity      17 non-null     int64  
 6   DayPart       17 non-null     int64  
 7   Lesson        17 non-null     int64  
 8   Grade         17 non-null     int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 1.3+ KB


Features

In [48]:
def feature_logs(df):

 # Taking the first character of the Gender column gives the deck, and mapping single characters to groups of decks; other decks will be NaN
    df['Gender'] = df['Gender'].map({'male': 'm', 'female': 'f'})

        # Limit numbers to those DayPart; other titles will be NaN
    df['DayPart'] = df['DayPart'].map({1: 1, 2: 2})

        # Change MotherTongue to numbers (est=1, rus=0)
    df['MotherTongue'] = df['MotherTongue'].map({'est': 1, 'rus': 0})

    return df


In [49]:
df = feature_logs(df) #Applay Features

Random sample of rows for checking

In [16]:
df.sample(n=5) #Random rows

,ID,Spec,MotherTongue,Gender,Age,Activity,DayPart,Lesson,Grade
7,26604,2,0,m,23.000,6,1.000,1,4
6,29133,1,1,m,20.000,14,1.000,1,5
9,26563,4,1,m,21.000,14,1.000,1,4
11,26789,3,1,m,22.000,12,2.000,0,0
16,23558,4,1,f,NaN,7,NaN,0,0


Drop column

In [50]:
train_labels = df['Grade'].copy()
df = df.drop('Grade', axis=1)

In [51]:
df.describe(include=np.number)

,ID,Spec,MotherTongue,Age,Activity,DayPart,Lesson
count,17.000,17.000,17.000,13.000,17.000,13.000,17.000
mean,27801.647,2.706,0.765,21.538,10.235,1.154,0.765
std,1758.647,2.469,0.437,1.984,5.166,0.376,0.437
min,23558.000,1.000,0.000,20.000,4.000,1.000,0.000
25%,26610.000,1.000,1.000,20.000,7.000,1.000,1.000
50%,26864.000,2.000,1.000,21.000,8.000,1.000,1.000
75%,29349.000,4.000,1.000,22.000,14.000,1.000,1.000
max,29723.000,11.000,1.000,26.000,23.000,2.000,1.000


Data clearning

In [52]:
df.isna().sum() #Columns with missing value

ID              0
Spec            0
MotherTongue    0
Gender          0
Age             4
Activity        0
DayPart         4
Lesson          0
dtype: int64

In [53]:
#Fill missing values with the modes
train_modes = df.mode().iloc[0]
train_modes

ID              23558
Spec            1.000
MotherTongue    1.000
Gender              m
Age            20.000
Activity        7.000
DayPart         1.000
Lesson          1.000
Name: 0, dtype: object

In [54]:

def fill_missing(df, modes):
    df = df.fillna(modes)
    return df

df = fill_missing(df, train_modes)

In [55]:
df

,ID,Spec,MotherTongue,Gender,Age,Activity,DayPart,Lesson
0,29292,1,1,f,20.000,23,1.000,1
1,26864,2,0,m,20.000,7,1.000,0
2,26861,2,0,m,21.000,18,1.000,1
3,26760,3,1,f,20.000,8,1.000,1
4,26544,4,1,m,21.000,9,1.000,1
5,29598,11,1,m,25.000,7,1.000,1
6,29133,1,1,m,20.000,14,1.000,1
7,26604,2,0,m,23.000,6,1.000,1
8,29346,1,1,m,20.000,4,1.000,1
9,26563,4,1,m,21.000,14,1.000,1


Additional converting

In [56]:
def feature_logs(df):

        # Change Gender to numbers (m=1, f=0)
    df['Gender'] = df['Gender'].map({'m': 1, 'f': 0})

    return df

In [57]:
df = feature_logs(df) #Applay Features

In [58]:
df.sample(n=5) #Random rows

,ID,Spec,MotherTongue,Gender,Age,Activity,DayPart,Lesson
10,26610,4,1,1,21.000,7,1.000,1
1,26864,2,0,1,20.000,7,1.000,0
15,29350,1,1,1,20.000,6,1.000,1
11,26789,3,1,1,22.000,12,2.000,0
13,29723,1,1,1,20.000,14,1.000,1


Min-max scaling

In [59]:
def scale_min_max(df, col_name, xmin, xmax):
    df[col_name] = (df[col_name] - xmin) / (xmax - xmin)
    return df

train_age_min = df['Age'].min()
train_age_max = df['Age'].max()
df = scale_min_max(df, 'Age', train_age_min, train_age_max)

df['Age'].describe()

count   17.000
mean     0.196
std      0.308
min      0.000
25%      0.000
50%      0.000
75%      0.167
max      1.000
Name: Age, dtype: float64

Dummy variables addition:

In [60]:
def add_dummies(df, cols):
    df = pd.get_dummies(df, columns=cols)
    return df

cols = ['Activity', 'Gender', 'Spec', 'ID']
df = add_dummies(df, cols)

print(df.columns, '\n', len(df.columns))


Index(['MotherTongue', 'Age', 'DayPart', 'Lesson', 'Activity_4', 'Activity_5',
       'Activity_6', 'Activity_7', 'Activity_8', 'Activity_9', 'Activity_12',
       'Activity_13', 'Activity_14', 'Activity_18', 'Activity_23', 'Gender_0',
       'Gender_1', 'Spec_1', 'Spec_2', 'Spec_3', 'Spec_4', 'Spec_11',
       'ID_23558', 'ID_26544', 'ID_26563', 'ID_26604', 'ID_26610', 'ID_26760',
       'ID_26789', 'ID_26861', 'ID_26864', 'ID_29133', 'ID_29292', 'ID_29346',
       'ID_29349', 'ID_29350', 'ID_29598', 'ID_29684', 'ID_29723'],
      dtype='object') 
 39


Data is procecced!

Data processing steps to the test set



In [61]:
test_proc = (test_df.pipe(feature_logs)
                    .drop(['DayPart', 'Lesson'], axis=1)
                    .pipe(fill_missing, train_modes)
                    .pipe(scale_min_max, 'Age', train_age_min, train_age_max)
                    .pipe(add_dummies, cols)
)

test_proc.sample(5)


,MotherTongue,Age,Grade,Activity_4,Activity_5,Activity_6,Activity_7,Activity_8,Activity_9,Activity_12,...,ID_26861,ID_26864,ID_29133,ID_29292,ID_29346,ID_29349,ID_29350,ID_29598,ID_29684,ID_29723
11,est,0.333,0,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
7,rus,0.500,4,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12,est,0.000,4,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
8,est,0.000,5,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,rus,0.167,3,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


Model training

In [62]:
device = 'cuda' if torch.cuda.is_available() else 'gpu'
device


'gpu'

In [63]:
train_modes = df.mode().iloc[0] #Fill missing values with the modes:
train_modes

MotherTongue        1
Age             0.000
DayPart         1.000
Lesson              1
Activity_4      False
Activity_5      False
Activity_6      False
Activity_7      False
Activity_8      False
Activity_9      False
Activity_12     False
Activity_13     False
Activity_14     False
Activity_18     False
Activity_23     False
Gender_0        False
Gender_1         True
Spec_1          False
Spec_2          False
Spec_3          False
Spec_4          False
Spec_11         False
ID_23558        False
ID_26544        False
ID_26563        False
ID_26604        False
ID_26610        False
ID_26760        False
ID_26789        False
ID_26861        False
ID_26864        False
ID_29133        False
ID_29292        False
ID_29346        False
ID_29349        False
ID_29350        False
ID_29598        False
ID_29684        False
ID_29723        False
Name: 0, dtype: object

Convert to tensor

In [64]:
x_data = torch.tensor(df.values, dtype=torch.float32)
y_data = torch.tensor(train_labels.values, dtype=torch.float32)

type(x_data), type(y_data)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.